# Xây dựng mô hình hồi quy dự đoán Doanh số của 1 dòng sản phẩm Khi có 1 dòng sản phẩm mới 

## EDA 

In [2]:
import pandas as pd
import pyodbc
from sklearn.linear_model import LinearRegression

server = 'WATERMELONX86'
database = 'DDSDA'
trusted_connection = 'yes'

conn_str = (
    f'DRIVER={{SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'Trusted_Connection={trusted_connection};'
)

conn = pyodbc.connect(conn_str)
sql_query = """
SELECT
    p.ProductLineId,
    SUM(f.Quantity) AS total_quantity,
	AVG(p.UnitPrice) as average_unit_price,
    AVG(f.Rating) AS average_rating,
	SUM(f.GrossIncome) as Total_income,
    SUM(f.Total) AS total_sales
FROM Fact_Sales f
INNER JOIN Dim_Product p ON f.Product_Key = p.Product_SK
GROUP BY
    p.ProductLineId
ORDER BY p.ProductLineId
"""
df = pd.read_sql(sql_query, conn)
conn.close()


C:\Users\water\AppData\Local\Temp\ipykernel_11744\3984690518.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


Data sau khi EDA

In [3]:
df

,ProductLineId,total_quantity,average_unit_price,average_rating,Total_income,total_sales
0,PD001,826,56.0149,6.901961,2183.3855,45851.0955
1,PD002,918,56.2830,6.968452,2557.7365,53712.4665
2,PD003,922,54.3010,7.110588,2464.0280,51744.5880
3,PD004,1005,55.2208,6.871839,2739.6805,57533.2905
4,PD005,959,54.9429,7.085465,2939.1710,61722.5910
5,PD006,962,54.5154,6.843575,2738.4205,57506.8305


## Xây dựng và kiểm tra mô hình 

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

features = df[[ 'total_quantity', 'average_unit_price', 'average_rating','Total_income']]
target = df['total_sales']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Khởi tạo mô hình và huấn luyện
model = LinearRegression()
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)
# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 1.6038697984605902e-05


Với Mean Squared nhỏ tiệm cận bằng 0 có thể thấy mô hình có ý nghĩa thống kê và chính xác cao khi dùng để đánh giá , dự đoán

## Dự đoán 

In [7]:
# Dữ liệu giả định cho dòng sản phẩm mới
new_ProductLine = [500, 70.2, 8.0, 2400.2] 
# Dự đoán 
sales_prediction = model.predict([new_ProductLine])
# In kết quả dự đoán
print('Dự đoán doanh số bán hàng cho dòng sản phẩm mới :', sales_prediction[0])

Dự đoán doanh số bán hàng cho dòng sản phẩm mới : 50404.22761535522


c:\Users\water\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
